# Deliverable 2: Determine Bias of Vine Reviews

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-24 06:01:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-05-24 06:01:30 (6.66 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

### Create DataFrames to match tables

In [5]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [6]:
# Filter data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
vine_helpful_votes_df = vine_df.filter("total_votes>=20")
vine_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [7]:
# Filter the new DataFrame and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_greater50_df = vine_helpful_votes_df.filter((vine_helpful_votes_df.helpful_votes / vine_helpful_votes_df.total_votes) >= 0.5) 
vine_greater50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [34]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_paid_df = vine_greater50_df.filter(vine_greater50_df["vine"] == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [36]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_unpaid_df = vine_greater50_df.filter(vine_greater50_df["vine"] == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

### Total Number of Reviews

In [37]:
# Determine the total number of reviews - TOTAL
total_reviews = vine_df.count() 
total_reviews

4751577

In [38]:
# Determine the total number of reviews - PAID
paid_reviews = vine_paid_df.count() 
paid_reviews

7

In [40]:
# Determine the total number of reviews - UNPAID
unpaid_reviews = vine_unpaid_df.count() 
unpaid_reviews

105979

### Total Number of 5-star reviews 

In [42]:
# Determine the number of 5-star reviews - PAID
paid_5star_reviews = vine_paid_df.filter(vine_paid_df["star_rating"] == 5).count()
paid_5star_reviews

0

In [31]:
# Determine the number of 5-star reviews - UNPAID
unpaid_5star_reviews = vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == 5).count() 
unpaid_5star_reviews

67580

### Total Percentage of 5-star reviews

In [25]:
# Determine the percentage of 5-star reviews - PAID
percentage_paid = (paid_5star_reviews/paid_reviews) *100
percentage_paid

0.0

In [26]:
# Determine the percentage of 5-star reviews - UNPAID
percentage_unpaid = (unpaid_5star_reviews/unpaid_reviews) *100
percentage_unpaid

63.767350135404186